imports

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.window import Window

read bronze sales table

In [0]:
spark=SparkSession.builder.appName("read bronze data").getOrCreate()
df_sales_bronze=spark.read\
    .format("delta")\
    .load("/Volumes/customer_360/customer_360_bronze/bronze_sales_volume")

INCREMENTAL

In [0]:
src_bronze_path="/Volumes/customer_360/customer_360_bronze/bronze_sales_volume"
silver_path="/Volumes/customer_360/customer_360_silver/silver_sales_volume"
if DeltaTable.isDeltaTable(spark, silver_path):
    bronze_table = DeltaTable.forPath(spark, silver_path)
    # Get max data_arrival_timestamp
    max_ts_row = bronze_table.toDF().select(max("data_arrival_timestamp")).collect()[0]
    max_ts = max_ts_row[0]  # None if table is empty
    if max_ts is None:
        print("Bronze table is empty. Will load all records.")
else:
    print("Bronze table not found. Will load all records.")
    max_ts = None  # first load

# Filter source for incremental load
if max_ts:
    df = df_sales_bronze.filter(col("data_arrival_timestamp") > max_ts)
else:
    df = df_sales_bronze  # first load, take all records

print(f"Number of records to load: {df.count()}")

drop duplicates

In [0]:
df=df.dropDuplicates(['order_id','data_arrival_timestamp'])


drop nulls

In [0]:
df = df.dropna(subset=["order_id", "order_date","ship_date","customer_id","product_id","sales","data_arrival_timestamp"],how='any')


fill nulls

In [0]:
df=df.fillna({
    "quantity":1,
    "discount":0,
    "profit":0
})

In [0]:
df.display()

INVALID CHECKS

In [0]:
df = df.filter(col("order_id").startswith("ORDER"))

In [0]:
df.display()

In [0]:
df = df.filter(col("ship_date") >= col("order_date"))

df = df.filter(col("order_date") <= current_date())

df = df.filter(col("data_arrival_timestamp") > col("order_date"))

df=df.filter((col("profit") <= col("sales")))



In [0]:
df.display()

In [0]:
df=df.withColumn("order_date", to_date(col("order_date"), "yyyy-MM-dd"))\
    .withColumn("ship_date", to_date(col("ship_date"), "yyyy-MM-dd"))

In [0]:
df = df.withColumn("is_deleted", lit(False))

In [0]:
df.display()

In [0]:
df = (
    df.withColumn("order_id", col("order_id").cast(StringType()))\
    .withColumn("order_date", to_date(col("order_date"), "yyyy-MM-dd"))\
    .withColumn("ship_date", to_date(col("ship_date"), "yyyy-MM-dd"))\
    .withColumn("ship_mode", col("ship_mode").cast(StringType()))\
    .withColumn("customer_id", col("customer_id").cast(StringType()))\
    .withColumn("product_id", col("product_id").cast(StringType()))\
    .withColumn("sales", col("sales").cast(DoubleType()))\
    .withColumn("quantity", col("quantity").cast(IntegerType()))\
    .withColumn("discount", col("discount").cast(DoubleType()))\
    .withColumn("profit", col("profit").cast(DoubleType()))\
    .withColumn("data_arrival_timestamp", to_timestamp(col("data_arrival_timestamp")))\
    .withColumn("is_deleted", lit(False).cast(BooleanType()))
)

In [0]:
# df_silver_product=spark.read\
#     .format("delta")\
#     .load("/Volumes/customer_360/customer_360_silver/silver_product_volume")

# df_silver_customer=spark.read\
#     .format("delta")\
#     .load("/Volumes/customer_360/customer_360_silver/silver_customer_volume")


# df = df.join(
#     df_silver_customer.select("customer_id"),
#     on="customer_id",
#     how="inner"
# )

# df = df.join(
#     df_silver_product.select("product_id"),
#     on="product_id",
#     how="inner"
# )

In [0]:
df = (
    df.withColumn("order_year", year("order_date"))
    .withColumn("order_month", month("order_date"))
)


In [0]:
df.write\
    .format("delta")\
    .mode("append")\
    .option("mergeSchema", "true")\
    .partitionBy("order_year", "order_month")\
    .save("/Volumes/customer_360/customer_360_silver/silver_sales_volume")


In [0]:
from datetime import datetime
from pyspark.sql.functions import max
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType

# Count records
records_count = df.count()

# max timestamp (only if rows exist)
max_data_ts_row = (
    df.select(max("data_arrival_timestamp")).collect()[0][0]
    if records_count > 0
    else None
)

# Use Python datetime for load_time
load_time = datetime.now()

# Define schema explicitly
schema = StructType([
    StructField("layer", StringType(), True),
    StructField("table_name", StringType(), True),
    StructField("load_time", TimestampType(), True),
    StructField("records_loaded", LongType(), True),
    StructField("max_data_timestamp", TimestampType(), True)
])

# Prepare audit data (even if 0 rows)
data = [("silver", "silver_sales", load_time, records_count, max_data_ts_row)]

# Create DataFrame
df_audit = spark.createDataFrame(data, schema)

# Append to audit table
df_audit.write.format("delta") \
    .mode("append") \
    .save("/Volumes/customer_360/audit/audit_volume/etl_audit")

print(f"Audit log updated successfully. Records loaded: {records_count}")
